In [1]:
## Cargo las librerias
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
 

##Cargo las librerias de omdb
import omdb
import requests

##Cargo librerias de scraping web
import re 
from bs4 import BeautifulSoup
import requests

##Para archivos y directorios
import io
import os
import chardet

##Librerias de tiempo
import time
from datetime import datetime
import time

##Libreria del zip
from zipfile import ZipFile

#Con esto podemos echar un vistazo a código HTML directamente en la Notebook
from IPython.display import HTML, display

##Librerías utilizadas para el muestreo de género de las peliculas
from collections import Counter
from random import uniform
from random import random
from bisect import bisect

###Librerias para el conteno de palabras (subtitulos)
from sklearn.feature_extraction.text import CountVectorizer
import nltk as nl
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import string as st
from textblob import TextBlob

##Separar training y test
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn import preprocessing
##Metricas
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, accuracy_score

##Modelos
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score


##Graficos
% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [73]:
#################################################################################################################################
##Importo los archivo
#################################################################################################################################
sub_text = pd.read_csv('./Subtitulos_limpios.csv', sep = ';')
#data_movies = pd.read_csv('./data_movies.csv', sep = ';')
data_movies = pd.read_csv('./data_movies_genero_V2.csv', sep = ';')



In [ ]:
##########################################################################################################
########################################## Trabajo con el género #########################################
##########################################################################################################

##1) calculo la frecuencia de los generos individuales
#Parto cada uno de los generos en una listas de listas (una lista de generos por pelicula)
generos = [i.split(', ') for i in data_movies['genre']]
##Creo una sola lista de las listas por peliculas
generos_lista = sum(generos, [])
##Hay en total 10.830 registros

##Calculo las probabilidades de ocurrencia cada uno de los generos
generos = Counter(generos_lista)
generos = [(i, generos[i] / len(generos_lista)) for i in generos]

##Funcion que toma los generos con sus probabilidades y eligue uno 
def weighted_choice(choices):
    values, weights = zip(*choices)
    total = 0
    cum_weights = []
    for w in weights:
        total += w
        cum_weights.append(total)
    x = random() * total
    i = bisect(cum_weights, x)
    return values[i]

In [ ]:
##A cada genero de las peliculas le pego su probabilidad
pos = [z[0] for z in generos]
list_generos=[]
for i in [i.split(', ') for i in data_movies['genre']]:
    list1 = []
    for j in i:
        list0 = (j, generos[pos.index(j)][1])
        list1.append(list0)
    list_generos.append(list1)

In [ ]:
##Muestreo los generos y se los agrego al dataset
data_movies['genero'] = [weighted_choice(i) for i in list_generos]

In [ ]:
###Compruebo que al menos el genero que muestreo este entre los generos posibles
genre = [i.split(', ') for i in data_movies['genre']]
genero = data_movies['genero']
sum([genero[i] in genre[i]  for i in range(len(genero))])
##Como la suma me da 4318 que es la cantidad total de registros del data set data_movie, esto quiere decir que esta correcto el genero muestreado

In [72]:
##################################################################################################################################
########## Arreglo el dataset data_movies
##################################################################################################################################
##Borro los campos que estan en NULL
#data_movies = data_movies.drop(['tomato_consensus', 'tomato_fresh','tomato_image','tomato_meter','tomato_rating','tomato_reviews','tomato_rotten','tomato_user_meter','tomato_user_rating','tomato_user_reviews'], axis=1)

##Reemplazo los N/A por nan
data_movies = data_movies.replace('N/A', np.nan)

##Transformo las fechas en date
data_movies.released = pd.to_datetime(data_movies.released)

## Elimino la palabra min a la Duracion
def intminutes(x):
    if pd.isnull(x):
        return np.nan
    x = str(x)
    y = x.replace('min', '').strip()
    return int(y)
data_movies.runtime = data_movies.runtime.apply(intminutes)

##Paso a entero el año
data_movies.year = data_movies.year.astype(int)

## Paso el imdbrating a float
data_movies.imdb_rating = data_movies.imdb_rating.astype(float)

##Le saco las comas a imdb_votes
def intvotes(x):
    if pd.isnull(x):
        return np.nan
    x = str(x)
    y = x.replace(',', '').strip()
    return int(y)
#data_movies.imdb_votes = data_movies.imdb_votes.apply(intvotes)

In [ ]:
##Guardo el data set data_movie con el genero muestreado
#data_movies.to_csv('data_movies_genero_V2.csv', sep=';', index = False)

In [61]:
###############################################################################################################
########################################## Trabajo con los subtitulos #########################################
###############################################################################################################
##Analizo el idioma del texto y muestro solamente los que no son ingles
idioma=[]
for i in sub_text.subtitulo:
    b = TextBlob(i)
    idio = b.detect_language()
    idioma.append(idio)
    if not(idio == 'en'):


21 sl
187 bg
216 el
318 es
366 bg
452 es
468 es
597 fr
689 iw
724 es
882 sw
944 pt
1001 es
1190 id
1225 zh-CN
1245 zh-CN
1290 zu
1407 ro
1444 it
1500 fr
1508 es
1568 es
1571 bg
1670 es
1726 ms
1730 pt
1765 id
1882 es
1944 es
2190 cs
2202 pl
2203 nl
2206 hu
2308 sr
2330 id
2371 iw
2448 es
2549 pt
2581 de
2674 es
2679 zh-CN
2700 la
3000 id
3064 zh-CN
3066 de


In [74]:
#sub_text.subtitulo[2679]
#data_movies.loc[data_movies.imdb_id == sub_text.imdb_id[2679],:]
#sub_text.imdb_id[724]        print(len(idioma)-1, idio)

##Borro los subtitulos que no estan en ingles
sub_text = sub_text.drop(sub_text.index[[21,187,216,318,366,452,468,597,689,724,882,944,1001,1190,1225,1245,1290,1407,1444,1500,1508,1568,1571,1670,1726,1730,1765,1882,1944,2190,2202,2203,2206,2308,2330,2371,2448,2549,2581,2674,2679,2700,3000,3064,3066]])

In [75]:
sub_text.head(50)[19:25]

,imdb_id,subtitulo
19,tt0164114,Oh man \n\n I am so wired \n More so than u...
20,tt0083943,SRT Reviewed TJK\n\nOne target left Rejoin ...
22,tt0395169,The latest ceasefire came too\nlate to save th...
23,tt1645080,Since the dawn\nof recorded history \n\nsometh...
24,tt1877832,\n\nPROFESSOR X The future \nA dark desolate...
25,tt0800039,Good for you Pete Good for you \n\n\nBILLY B...


In [76]:
##Reseteo el indice
sub_text = sub_text.reset_index(drop=True)

In [77]:
sub_text.head(50)[8:25]

,imdb_id,subtitulo
8,tt0099700,Stop the music \n\n What's up duck \n What'...
9,tt1193138,This is what I get in return\nfor 30 years of ...
10,tt0815178,That old bitch \n\nI'm serious \nIf I have to ...
11,tt1371111,Lonesome night \n\nBabbits bawlin' wind bitin...
12,tt0420238,Once upon a time \nthere was a brave little mo...
13,tt0468526,Leave a message we'll call you back \n\nHey ...
14,tt2226519,Wait Come back \n\nNo don't do this \n\nAah ...
15,tt0217630,I think this might be it \nPaul Good luck \n\...
16,tt0438488,\n\n\nMarcus \n\nHow are you \n\nIt's been an ...
17,tt3787590,Well it's gonna\nbe a big night right \n\nI ...


In [78]:
##Limpio los subtitulos
for i in range(sub_text.shape[0]):
    numeros = re.compile('\d*')
    comillas2 = re.compile("''")
    comillas = re.compile("'")
    espacios = re.compile("\s{2,}")
    enter = re.compile("\n{2,}")
    solo_enter = re.compile ("\n")
    
    texto = sub_text.iloc[i, 1]
    texto = numeros.sub('', texto)
    texto = comillas2.sub('', texto)
    texto = comillas.sub('', texto)
    texto = espacios.sub(' ', texto)
    texto = enter.sub('\n', texto)
    texto = solo_enter.sub(' ', texto).strip()
    sub_text.iloc[i, 1] = texto

In [88]:
sub_text[sub_text.index==8]

,imdb_id,subtitulo
8,tt0099700,Stop the music Whats up duck Whats up Ill tell...


In [122]:
sub_text.shape

(3295, 2)

In [134]:
hora_ini = datetime.now()

##Funciones para tokenizar y hacer stemmer
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nl.word_tokenize(text)
    tokens = [i for i in tokens if i not in st.punctuation]
    stems = stem_tokens(tokens, stemmer)
    return stems

##Hago countvectorizer con los subtitulos y creo un dataframe con los mismos
cv = CountVectorizer(tokenizer=tokenize, stop_words='english', lowercase=True, min_df=10, max_df=0.8)
#cv = CountVectorizer(stop_words='english', lowercase=True, min_df=10, max_df=0.90)

data = cv.fit_transform(sub_text.subtitulo).todense()

print('Duration: {}'.format(datetime.now() - hora_ini))


Duration: 0:18:55.516544


In [135]:
len(cv.get_feature_names())

18454

In [137]:
#Genero un data frame con las palabras
columns = [c for c in cv.get_feature_names()]
subdf = pd.DataFrame(data, columns=columns)
subdf.shape

subdf.iloc[8,:].sum()

2005

In [138]:
### Le pego el imdb_id
subdf['imdb_id'] = sub_text['imdb_id']

In [142]:
subdf.loc[subdf.index==8, 'music']

8    5
Name: music, dtype: int64

In [143]:
##Le pego los datos del target
movie_completo = pd.merge(left = subdf, right = data_movies, on='imdb_id')

In [145]:
movie_completo.shape

(3295, 18480)

In [146]:
##Guardo el data set
movie_completo.to_csv('4_movie_completo.csv', sep=';', index = False)

In [152]:
movie_completo.shape

(3295, 18480)